In [1]:
setwd('/lustre/scratch117/cellgen/team297/kt16/newcastle_covid')
library(ggplot2)

In [2]:
library(dplyr)
makeGeneList <- function(filename){
    gl <- readr::read_tsv(filename)
    # gl <- readr::read_csv(filename)
    y <- grepl('^RPS|^RPL|^MRPL|^MRPS|^MT-', gl$X1)
    gl <- gl[!y, ]
    gl <- gl %>% dplyr::select(X1, logfoldchanges , pvals)
    gl$neglog10pval <- -log10(gl$pvals)
    rank <- unlist(gl$neglog10pval*sign(gl$logfoldchanges))
    rank[rank == Inf] = 300
    rank[rank == -Inf] = -300
    names(rank) <- gl$X1
    rank <- rev(sort(rank))
    gl <- rank
}


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [3]:
celltypes = c('B_exhausted', 'B_immature', 'B_naive', 'B_non-switched_memory', 'B_switched_memory', 'Plasmablast', 'Plasma_cell')
groups = c('Critical', 'Severe', 'Moderate', 'Mild', 'Asymptomatic')

In [4]:
comparisons = list()
for (c in celltypes){
    for (g in groups){
        comparisons[[c]][[g]] = makeGeneList(paste0('DEG/wilcoxon/',g,'_',c, '_wilcoxon_v2.txt'))
    }
}

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  X1 = col_character(),
  scores = col_double(),
  logfoldchanges = col_double(),
  pvals = col_double(),
  pvals_adj = col_double()
)

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  X1 = col_character(),
  scores = col_double(),
  logfoldchanges = col_double(),
  pvals = col_double(),
  pvals_adj = col_double()
)

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  X1 = col_character(),
  scores = col_double(),
  logfoldchanges = col_double(),
  pvals = col_double(),
  pvals_adj = col_double()
)

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  X1 = col_character(),
  scores = col_double(),
  logfoldchanges = col_double(),
  pvals = col_double(),
  pvals_adj = col_double()
)

Warning message:
“Missing column names filled in: 'X1' [

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  X1 = col_character(),
  scores = col_double(),
  logfoldchanges = col_double(),
  pvals = col_double(),
  pvals_adj = col_double()
)

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  X1 = col_character(),
  scores = col_double(),
  logfoldchanges = col_double(),
  pvals = col_double(),
  pvals_adj = col_double()
)

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  X1 = col_character(),
  scores = col_double(),
  logfoldchanges = col_double(),
  pvals = col_double(),
  pvals_adj = col_double()
)

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  X1 = col_character(),
  scores = col_double(),
  logfoldchanges = col_double(),
  pvals = col_double(),
  pvals_adj = col_double()
)

Warning message:
“Missing column names filled in: 'X1' [

In [5]:
library(fgsea)

Loading required package: Rcpp

Warning message:
“package ‘Rcpp’ was built under R version 3.6.3”


In [6]:
h <- as.list(kelvinny::parse_gmt("datasets/h.all.v7.2.symbols.gmt"))

In [7]:
res = list()
for (c in celltypes){
    for (g in groups){
        res[[c]][[g]] <- fgsea(pathways = h, stats = comparisons[[c]][[g]], nperm = 10000, minSize = 0, maxSize =1000)
    }
}

Warning message in fgsea(pathways = h, stats = comparisons[[c]][[g]], nperm = 10000, :
“There are ties in the preranked stats (25.06% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
Warning message in fgsea(pathways = h, stats = comparisons[[c]][[g]], nperm = 10000, :
“There are ties in the preranked stats (24.7% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
Warning message in fgsea(pathways = h, stats = comparisons[[c]][[g]], nperm = 10000, :
“There are ties in the preranked stats (22.67% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
Warning message in fgsea(pathways = h, stats = comparisons[[c]][[g]], nperm = 10000, :
“There are ties in the preranked stats (26.08% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
Warning message in fgsea(pathways = h, stats = comparison

In [8]:
for (c in celltypes){
    for (g in groups){
        res[[c]][[g]] <- as.data.frame(res[[c]][[g]])
    }
}

In [9]:
result = res

# names(result) <- gsub('.csv', "",files)
for(i in 1:length(comparisons)){
    result[[i]] <- lapply(result[[i]], function(x){
    x$ranking <- -log10(x$pval)*sign(x$NES) 
    x <- x[order(x$pathway), ]
    return(x)
})
}

In [10]:
result <- lapply(result ,function(x){
    x[['Critical']]$group = "Critical"
    x[['Severe']]$group = "Severe"
    x[['Moderate']]$group = "Moderate"
    x[['Mild']]$group = "Mild"
    x[['Asymptomatic']]$group = "Asymptomatic"
    return(x)
})

In [11]:
result2 <- lapply(result, function(x) {
    y <- do.call(rbind, x)
    return(y)
})

In [12]:
saveRDS(result2, 'DEG/wilcoxon/Bcell_GSEA_results_v2.RDS')

In [13]:
plotGSEA_Hallmark <- function(gsea, group_ref = NULL, cols = NULL, newlabels = NULL, keep_significant_only = FALSE) {
    require(ggplot2)
    gsea$NES[which(is.na(gsea$NES))] <- 0
    gsea$pval[which(is.na(gsea$pval))] <- 1
    gsea$padj[which(is.na(gsea$padj))] <- 1
    gsea$ranking[which(is.na(gsea$ranking))] <- 0
    gsea <- gsea[order(gsea$ranking),]      
    gsea_spl <- split(gsea, gsea$group)
    if(!is.null(group_ref)){
        gsea_spl[[group_ref]] <- gsea_spl[[group_ref]][order(gsea_spl[[group_ref]]$ranking),]
        gsea_spl[[group_ref]]$ranking <- gsea_spl[[group_ref]]$ranking*999
    } else {
        gsea_spl[[2]] <- gsea_spl[[2]]$ranking*999
    }
    names(gsea_spl) <- NULL

    gsea <- do.call(rbind, gsea_spl)
    
    if (keep_significant_only){
        gseax <- split(gsea, gsea$pathway)
        for (i in 1:length(gseax)){
            if (all(gseax[[i]]$pval >= 0.05)|all(gseax[[i]]$padj >=0.25)){
                gseax[[i]] <- NA        
            }
        }
        gseax <- gseax[!is.na(gseax)]
        gsea <- do.call(rbind, gseax)
    }
    gsea <- gsea[order(gsea$ranking), ]
    gsea$pathway <- gsub("HALLMARK_|", "", gsea$pathway)
    gsea$group[which(gsea$pval >= 0.05 & gsea$padj >= 0.25)] <- 'NotSig'
    gsea$group[which(gsea$pval < 0.05 & gsea$padj >= 0.25)] <- 'NotSig'
    gsea$group[which(gsea$pval >= 0.05)] <- 'NotSig'
    gsea$group <- factor(gsea$group, levels = c('NotSig', 'Asymptomatic', 'Mild', 'Moderate', 'Severe', 'Critical'))

    x_lim_min <- abs(ceiling(min(-log10(gsea$padj))))
    x_lim_max <- abs(ceiling(max(-log10(gsea$padj))))
        
    if(x_lim_min > x_lim_max){
        xval1 <- x_lim_min * -1
        xval2 <- x_lim_min
    } else {
        xval1 <- x_lim_max * -1
        xval2 <- x_lim_max
    }

    if(!is.null(cols)){
        gg_color_hue <- function(n) {
            hues = seq(15, 375, length = n + 1)
            hcl(h = hues, l = 65, c = 100)[1:n]
        }
        cols. = gg_color_hue(dplyr::n_distinct(gsea$group, na.rm = TRUE))
    } else {
        cols. = cols
    }    
    
    g <- ggplot(gsea, aes(x = -log10(padj)*sign(NES), y = reorder(pathway, ranking), col = group, size = abs(NES))) + 
        geom_point() + 
        labs(x = expression(paste("Signed", " -log" ["10"], "adjusted pval")), y = "Hallmarks") +
        theme_bw() +
        geom_vline(xintercept = 0) +
        geom_vline(xintercept = -log10(0.25)) +
        geom_vline(xintercept = -log10(0.25)*-1) +
        xlim(xval1, xval2) +
        scale_size_area(oob = scales::squish, max_size = 3, limits = c(0,2)) +
        theme(panel.grid.major = element_blank(), 
            panel.grid.minor = element_blank(), 
            panel.background = element_blank(), 
            axis.line = element_blank(), 
            axis.ticks = element_blank())
    
    g$data <- g$data[order(g$data$group, na.last = TRUE), ]
    return(g)
}

In [14]:
kelvinny::dirCreate('figures/gsea')

In [15]:
for(i in 1:length(result2)){
p <- plotGSEA_Hallmark(result2[[i]], group_ref = "Moderate") + scale_color_manual(values = c('#e7e7e7' , '#fed976' ,'#fd8d3c' ,'#e31a1c' ,'#800026' ,'#252525'), drop=FALSE)
ggsave(paste0('figures/gsea/',names(result2)[i],"_v2.pdf"), plot = p, w = 6, useDingbats = FALSE)
}

Saving 6 x 6.67 in image

Saving 6 x 6.67 in image

Saving 6 x 6.67 in image

Saving 6 x 6.67 in image

Saving 6 x 6.67 in image

Saving 6 x 6.67 in image

Saving 6 x 6.67 in image



In [16]:
names(result2)

[1] "B_exhausted"           "B_immature"            "B_naive"              
[4] "B_non-switched_memory" "B_switched_memory"     "Plasmablast"          
[7] "Plasma_cell"

In [17]:
pp <- c(6, 5, 13, 7, 11, 15, 10)
for(i in 1:length(result2)){
    p <- plotGSEA_Hallmark(result2[[i]], group_ref = "Moderate", keep_significant_only = TRUE) + scale_color_manual(values = c('#e7e7e7','#fed976' ,'#fd8d3c' ,'#e31a1c' ,'#800026' ,'#252525'), drop=FALSE)
    ggsave(paste0('figures/gsea/wilcoxon_',names(result2)[i],"_v3.pdf"), plot = p, w = 6, h = pp[i]/3, useDingbats = FALSE)
}

In [18]:
sessionInfo()

R version 3.6.1 (2019-07-05)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.5 LTS

Matrix products: default
BLAS/LAPACK: /nfs/users/nfs_k/kt16/miniconda2/envs/dandelion/lib/R/lib/libRblas.so

locale:
 [1] LC_CTYPE=en_GB.utf8       LC_NUMERIC=C             
 [3] LC_TIME=en_GB.utf8        LC_COLLATE=en_GB.utf8    
 [5] LC_MONETARY=en_GB.utf8    LC_MESSAGES=en_GB.utf8   
 [7] LC_PAPER=en_GB.utf8       LC_NAME=C                
 [9] LC_ADDRESS=C              LC_TELEPHONE=C           
[11] LC_MEASUREMENT=en_GB.utf8 LC_IDENTIFICATION=C      

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] fgsea_1.12.0  Rcpp_1.0.5    dplyr_1.0.0   ggplot2_3.3.2

loaded via a namespace (and not attached):
 [1] bitops_1.0-6                matrixStats_0.57.0         
 [3] fs_1.5.0                    doMC_1.3.7                 
 [5] usethis_1.6.3               devtools_2.3.2             
 [7] RColorBrewer_